In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3216421/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
15,model_2_1_12,0.904256,0.906839,0.795221,0.854042,0.105961,0.099144,0.220406,0.156209,"Hidden Size=[15, 7], regularizer=0.1, learning..."
16,model_2_1_11,0.903651,0.911267,0.803575,0.860327,0.106630,0.094432,0.211414,0.149482,"Hidden Size=[15, 7], regularizer=0.1, learning..."
17,model_2_1_13,0.903543,0.908166,0.786490,0.850608,0.106749,0.097732,0.229803,0.159883,"Hidden Size=[15, 7], regularizer=0.1, learning..."
18,model_2_1_14,0.900764,0.907999,0.762364,0.839103,0.109825,0.097909,0.255770,0.172197,"Hidden Size=[15, 7], regularizer=0.1, learning..."
19,model_2_1_15,0.899875,0.882950,0.769735,0.829404,0.110809,0.124568,0.247837,0.182577,"Hidden Size=[15, 7], regularizer=0.1, learning..."
...,...,...,...,...,...,...,...,...,...,...
141,model_4_9_8,0.852822,0.866548,0.770786,0.848373,0.162882,0.219973,0.291121,0.253455,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
142,model_2_1_4,0.852584,0.890075,0.795843,0.845512,0.163146,0.116985,0.219736,0.165338,"Hidden Size=[15, 7], regularizer=0.1, learning..."
143,model_4_9_7,0.852527,0.866763,0.770816,0.848498,0.163209,0.219619,0.291083,0.253246,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
145,model_4_9_6,0.852221,0.867010,0.770748,0.848601,0.163547,0.219211,0.291170,0.253075,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
